## Download builded OpenCV with gpu support
The building opencv from sources is required in order to use DNN module with GPU. That step was alreayd done and using the cell below, it will copy the required files to run Opencv DNN with GPU in the colab environment

In [ ]:
# download opencv
!gdown --id 1-0mj_acbT7LuV56ktYHjHVCp-kUaHsK8

Downloading...
From: https://drive.google.com/uc?id=1-0mj_acbT7LuV56ktYHjHVCp-kUaHsK8
To: /content/cv2.cpython-36m-x86_64-linux-gnu.so
973MB [00:04, 219MB/s]


## Download yolo3 configuration, weights, labels
This cell downloads to the colab environment the yolo3 detector from the minor group.

In [ ]:
# download yolo cfg
!gdown --id 1W8FO1appY9HXIT-MsAUV-bbYpXRpX9zr
# download yolo weights
!gdown --id 15PMgapgWLettfzXANaTcDZBOPpNkJ7q6
# download yolo labels
!gdown --id 1HFwN9JLLgEMvg7YlZKiPE-bSOtZSb5r1

Downloading...
From: https://drive.google.com/uc?id=1W8FO1appY9HXIT-MsAUV-bbYpXRpX9zr
To: /content/yolov3_custom_C4_C3_C4D_C3D_I.cfg
100% 8.33k/8.33k [00:00<00:00, 15.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=15PMgapgWLettfzXANaTcDZBOPpNkJ7q6
To: /content/yolov3_custom_C3_C4_C3D_C4D_I.weights
246MB [00:01, 162MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HFwN9JLLgEMvg7YlZKiPE-bSOtZSb5r1
To: /content/obj.names
100% 16.0/16.0 [00:00<00:00, 14.1kB/s]


# Check installation
This cell shold execute without problems and if 4.5.0-dev is printed for opencv version everything is ok.

In [ ]:
import numpy as np
import time
import cv2
#from google.colab.patches import cv2_imshow 

cv2.__version__

'4.5.0-dev'

# Setting paths to yolo detector
Set the required paths to the labels map, the configuration file and weights

In [ ]:
#Yolo3 minor
labelsPath='./obj.names'
weightsPath = './yolov3_custom_C3_C4_C3D_C4D_I.weights'  # damages paths
configPath = './yolov3_custom_C4_C3_C4D_C3D_I.cfg'

In [ ]:
ls ./

cv2.cpython-36m-x86_64-linux-gnu.so  sample_data/
drive/                               yolov3_custom_C3_C4_C3D_C4D_I.weights
obj.names                            yolov3_custom_C4_C3_C4D_C3D_I.cfg


## NN parameters
These parameters can be changed and they control how much results are displayed and the confidence threshold for the detector

In [ ]:
# NN params
confidence_param = 0.35
thresh_param = 0.25

## Initialize the neural network
loads the yolo configuration into the working environment

In [ ]:
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
LABELS = open(labelsPath).read().strip().split("\n")

# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

[INFO] loading YOLO from disk...


# Define function for performing detection
This method takes as input image and optional flag - if true, information about each detection will be logged (confidence, labelId, bounding box coordinates

In [ ]:
def detect_from_image(frame, verbose = False):
  try:            
      (H, W) = frame.shape[:2]
      blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),	swapRB=False, crop=False)
      net.setInput(blob)
      #start = time.time()
      layerOutputs = net.forward(ln)
      #end = time.time()
      #elap = (end - start)
      #print("[INFO] single frame took {:.4f} seconds".format(elap))
      # initialize our lists of detected bounding boxes, confidences,
      # and class IDs, respectively
      boxes = []
      confidences = []
      classIDs = []
  # loop over each of the layer outputs
      for output in layerOutputs:
          # loop over each of the detections
          for detection in output:
              # extract the class ID and confidence (i.e., probability)
              # of the current object detection
              scores = detection[5:]
              classID = np.argmax(scores)
              confidence = scores[classID]

              # filter out weak predictions by ensuring the detected
              # probability is greater than the minimum probability
              if confidence > confidence_param:
                  # scale the bounding box coordinates back relative to
                  # the size of the image, keeping in mind that YOLO
                  # actually returns the center (x, y)-coordinates of
                  # the bounding box followed by the boxes' width and
                  # height
                  box = detection[0:4] * np.array([W, H, W, H])
                  (centerX, centerY, width, height) = box.astype("int")

                  # use the center (x, y)-coordinates to derive the top
                  # and and left corner of the bounding box
                  x = int(centerX - (width / 2))
                  y = int(centerY - (height / 2))

                  # update our list of bounding box coordinates,
                  # confidences, and class IDs
                  boxes.append([x, y, int(width), int(height)])
                  confidences.append(float(confidence))
                  classIDs.append(classID)
                  if verbose :
                    print("Score:{} ClassId:{} Confidence:{} boxes:{}".format(scores,classID, confidence, [x, y, int(width), int(height)]))


  # apply non-maxima suppression to suppress weak, overlapping
  # bounding boxes
      idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence_param,
                              thresh_param)

      # ensure at least one detection exists
      if len(idxs) > 0:
          # loop over the indexes we are keeping
          for i in idxs.flatten():
              # extract the bounding box coordinates
              (x, y) = (boxes[i][0], boxes[i][1])
              (w, h) = (boxes[i][2], boxes[i][3])

              # draw a bounding box rectangle and label on the frame
              color = [int(c) for c in COLORS[classIDs[i]]]
              cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
              text = "{}: {:.4f}".format(LABELS[classIDs[i]],
                                        confidences[i])
              cv2.putText(frame, text, (x, y - 5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)       
            # some information on processing single frame
  except Exception as e:
    print("something happened: ", e)
      
  return frame

## Download test image and video

In [ ]:
# download image
!gdown --id 1AQiGNNsVJo4NVZlj0L0s-qmHRKEpOwSv
# download video
!gdown --id 1UUp4aGyu8gips1hjIGc1DANGdP8pCu8k

Downloading...
From: https://drive.google.com/uc?id=1AQiGNNsVJo4NVZlj0L0s-qmHRKEpOwSv
To: /content/1.png
100% 373k/373k [00:00<00:00, 59.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1UUp4aGyu8gips1hjIGc1DANGdP8pCu8k
To: /content/inspc.mpg
695MB [00:13, 51.5MB/s]


# Testing
The first cell tests the detector with the downloaded image
The second cell tests the detector with the downloaded video and saves it after it is done reading.

In [ ]:
image_path='./corrosion.JPG'
image_path='./crackHd2.png'

In [ ]:
# read input image
image_path='./corrosion.JPG'  # you can change the path to any image, if you upload different file
image = cv2.imread(image_path)
# process image
processed = detect_from_image(image, True)
# display
cv2_imshow(image)

### Note
This script has few variable that can be changed
* width & height - desired output for the resulting video
* count - the number of frames to read from the video ( this might be needed in the colab environment because of memory issues, lower the number if there are any)
* logging apears to take up a lot of ram as well, consider commenting out the inference speed timer & print; also set detect_from_image(..., verbose=False) verbose prints bounding boxes, label and confidence for detections

In [ ]:
print("reading video")
cam = cv2.VideoCapture('inspc.mpg')
#ret, frame = cam.read()
vid_arr = []
frame_counter = 0
# scale_percent = 40 # percent of original size
# width = int(frame.shape[1] * scale_percent / 100)
# height = int(frame.shape[0] * scale_percent / 100)
#width = 1920
#height = 1080
# dim = (width, height)
# resize image

counter1000frames = 1
#count = 1000
while cam.isOpened():
    ret, frame = cam.read()
    #frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    height, width, layers = frame.shape

    frame_counter+=1
    if frame_counter == 5:
      frame = detect_from_image(frame, True)
      vid_arr.append(frame)
      frame_counter = 0
    if len(vid_arr) >= 5000:
    #   counter1000frames +=5
       break
name = 'infered' + str(counter1000frames) +'.avi'
out = cv2.VideoWriter(name, cv2.VideoWriter_fourcc(*'DIVX'), 15, (width, height))
for i in range(len(vid_arr)):
    out.write(vid_arr[i])
out.release()
frame_counter = 0
# vid_arr = None
# vid_arr = []
print("saved {}" ,name)


